# Setup

In [1]:
from os import path
from typing import Optional

__DIR__ = globals()['_dh'][0]
data_dir = path.relpath(path.join(__DIR__, "..", "_data"))

In [2]:
# Settings
_colab_install = True
_pm_log_sections = False
_testing = True

# Parameters
dataset = path.join(data_dir, "wiki", "20220301.en.1gb")

base_model = "bert-base-cased"
max_length = 128
vocab_size = 20_000

tokenize_params = dict(batched=True, num_proc=4)
tokenizer_dir = path.join(data_dir, "pretrain", "tokenizer")

mlm_probability = 0.15
bert_config = dict()
training_args = dict(
    optim = "adamw_torch",
    num_train_epochs = 3,
    per_device_train_batch_size = 64,
    eval_accumulation_steps = 10,
    evaluation_strategy = "steps",
    logging_steps = 5000,
    save_steps = 5000,
    save_total_limit = 5,
)
max_eval_samples: Optional[int] = 2000
model_dir = path.join(data_dir, "pretrain", "model")

In [3]:
if _testing:
    dataset = path.join(data_dir, "wiki", "20220301.en.test")

    training_args.update(dict(
        max_steps = 3,
        logging_steps = 1,
    ))

    max_eval_samples = 1000

## Process settings / parameters

In [4]:
from pprint import pprint
from collections import OrderedDict

if _colab_install:
    try:
        import google.colab
        
        colab_install_script = path.join(__DIR__, "..", "colab_install.sh")

        if not path.isfile(colab_install_script):
            script_url = "https://raw.githubusercontent.com/yenson-lau/pii-remediation/main/colab_install.sh"
            !wget $script_url -O $colab_install_script

        !bash $colab_install_script

    except ModuleNotFoundError:
        pass

if _pm_log_sections:
    def pm_log_section(message):
        print(f"\n[===== {message} =====]\n")
else:
    def pm_log_section(message):
        return

if _testing:
    pm_log_section("Running on testing mode")

config = OrderedDict(
    dataset = dataset,

    base_model = base_model,
    max_length = max_length,
    vocab_size = vocab_size,

    tokenize_params = tokenize_params,
    tokenizer_dir = tokenizer_dir,

    mlm_probability = mlm_probability,
    bert_config = bert_config,
    training_args = training_args,
    max_eval_samples = max_eval_samples,
    model_dir = model_dir,
)

print(f"{'TESTING ' if _testing else ''}Parameters:")
pprint(config, indent=2)

TESTING Parameters:
OrderedDict([ ('dataset', '../_data/wiki/20220301.en.test'),
              ('base_model', 'bert-base-cased'),
              ('max_length', 128),
              ('vocab_size', 20000),
              ('tokenize_params', {'batched': True, 'num_proc': 4}),
              ('tokenizer_dir', '../_data/pretrain/tokenizer'),
              ('mlm_probability', 0.15),
              ('bert_config', {}),
              ( 'training_args',
                { 'eval_accumulation_steps': 10,
                  'evaluation_strategy': 'steps',
                  'logging_steps': 1,
                  'max_steps': 3,
                  'num_train_epochs': 3,
                  'optim': 'adamw_torch',
                  'per_device_train_batch_size': 64,
                  'save_steps': 5000,
                  'save_total_limit': 5}),
              ('max_eval_samples', 1000),
              ('model_dir', '../_data/pretrain/model')])


# Load dataset

In [5]:
from datasets import Dataset, load_dataset

pm_log_section("Loading dataset")

ds_dir = dataset
dataset = dict()
for split in ["train", "val"]:
    data_file = path.join(ds_dir, f"{split}_data.json")
    if not path.isfile(data_file):  data_file += ".gz"
    dataset[split] = load_dataset("json", data_files=data_file, field="data")["train"]

    if ((split != "train") 
        and (max_eval_samples is not None) 
        and (len(dataset[split]) > max_eval_samples)):

        dataset[split] = dataset[split].select(range(max_eval_samples))

Using custom data configuration default-e5574e295c9a56a3
Reusing dataset json (/Users/yenson/.cache/huggingface/datasets/json/default-e5574e295c9a56a3/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-e7bfbf373754ac09
Reusing dataset json (/Users/yenson/.cache/huggingface/datasets/json/default-e7bfbf373754ac09/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/1 [00:00<?, ?it/s]

# Tokenization

In [6]:
from transformers import BertTokenizerFast

pm_log_section("Tokenizing")

In [7]:
tokenizer = (BertTokenizerFast
                .from_pretrained(base_model)
                .train_new_from_iterator(dataset["train"]["text"], vocab_size))
tokenizer.model_max_length = max_length

tokenizer.save_pretrained(tokenizer_dir);

In [8]:
tokenize_function = lambda ex: tokenizer(ex["text"], truncation=True)

tokenized_dataset = {
    k: v.map(tokenize_function, remove_columns = list(v.features), **tokenize_params)
    for k, v in dataset.items()
}

#0:   0%|          | 0/3 [00:00<?, ?ba/s]

#1:   0%|          | 0/3 [00:00<?, ?ba/s]

#2:   0%|          | 0/3 [00:00<?, ?ba/s]

#3:   0%|          | 0/3 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

# Train masked language model

In [9]:
import numpy as np
from transformers import (BertConfig,
                          BertForMaskedLM,
                          DataCollatorForLanguageModeling,
                          Trainer,
                          TrainingArguments)

pm_log_section("Training MLM")

data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer,
                                                mlm_probability = mlm_probability)

bert_config = BertConfig(vocab_size = tokenizer.vocab_size, **bert_config)
model = BertForMaskedLM(config = bert_config)

training_args = TrainingArguments(output_dir = model_dir,
                                  overwrite_output_dir = True,
                                  **training_args)

# def compute_metrics(eval_preds):
#     idxs0, idxs1 = np.where(eval_preds.label_ids!=-100)

#     preds = np.argmax(eval_preds.predictions[idxs0, idxs1, :], axis=-1)
#     labels = eval_preds.label_ids[idxs0, idxs1]

#     acc = (preds==labels).sum()/len(preds)

#     return {"accuracy": acc}

In [11]:
trainer = Trainer(model = model,
                  args = training_args,
                  data_collator = data_collator,
                  train_dataset = tokenized_dataset["train"],
                  eval_dataset=tokenized_dataset["val"])

trainer.train()
trainer.save_model(model_dir)

max_steps is given, it will override any value given in num_train_epochs
***** Running training *****
  Num examples = 8788
  Num Epochs = 1
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 3


  0%|          | 0/3 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
***** Running Evaluation *****
  Num examples = 998
  Batch size = 8


{'loss': 10.1149, 'learning_rate': 3.3333333333333335e-05, 'epoch': 0.01}


  0%|          | 0/125 [00:00<?, ?it/s]

...

{'loss': 9.5463, 'learning_rate': 0.0, 'epoch': 0.02}


  0%|          | 0/125 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ../_data/pretrain/model
Configuration saved in ../_data/pretrain/model/config.json


{'eval_loss': 9.416359901428223, 'eval_runtime': 20.1873, 'eval_samples_per_second': 49.437, 'eval_steps_per_second': 6.192, 'epoch': 0.02}
{'train_runtime': 75.0642, 'train_samples_per_second': 2.558, 'train_steps_per_second': 0.04, 'train_loss': 9.775258700052897, 'epoch': 0.02}


Model weights saved in ../_data/pretrain/model/pytorch_model.bin


In [12]:
pm_log_section("Finished pretraining!")

In [13]:
trainer.evaluate()  # 9.39675235748291

***** Running Evaluation *****
  Num examples = 998
  Batch size = 8


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 9.39639663696289,
 'eval_runtime': 20.0376,
 'eval_samples_per_second': 49.806,
 'eval_steps_per_second': 6.238,
 'epoch': 0.02}